In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

In [3]:
torch.cuda.is_available()

True

In [18]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
device=torch.device('cpu')

In [19]:
device

device(type='cuda')

In [20]:
batch_size=64
num_classes=10 ## CIFAR-10 has 10 classes
lr=0.001
num_epochs=20

In [21]:
## Transforms that we need to implement on the image are: Resize, convert to Tensors, Normalize

transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                        std=[0.2023, 0.1994, 0.2010])
])

In [22]:
train=torchvision.datasets.CIFAR10(root='./data',
                                  train=True,
                                  transform=transform,
                                  download=True)

Files already downloaded and verified


In [23]:
test=torchvision.datasets.CIFAR10(root='./data',
                                  train=False,
                                  transform=transform,
                                  download=True
)

Files already downloaded and verified


In [24]:
## Data Loader

train_loader=torch.utils.data.DataLoader(dataset=train,
                                         batch_size=batch_size,
                                         shuffle=True)

test_dataloader=torch.utils.data.DataLoader(dataset=test,
                                           batch_size=batch_size,
                                           shuffle=True)

In [25]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv_layer1=nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2=nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.maxpool1=nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_layer3=nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4=nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.maxpool2=nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1=nn.Linear(1600,128)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(128,num_classes)


    def forward(self,x):
        out=self.conv_layer1(x)
        out=self.conv_layer2(out)
        out=self.maxpool1(out)
        out=self.conv_layer3(out)
        out=self.conv_layer4(out)
        out=self.maxpool2(out)
        out = out.reshape(out.size(0), -1)
        out=self.fc1(out)
        out=self.relu(out)
        out=self.fc2(out)
        return out

In [26]:
model=CNN(num_classes)
model=model.to(device)

In [27]:
criterion=nn.CrossEntropyLoss()

In [28]:
optimizer=torch.optim.SGD(model.parameters(), lr=lr, weight_decay = (0.005), momentum=0.9)

In [29]:
total_step=len(train_loader)

In [30]:
start=time.time()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images=images.to(device)
        labels=labels.to(device)
        output=model(images)
        loss=criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
end=time.time()
print(f"The duration is {end-start}")

Epoch [1/20], Loss: 1.1809
Epoch [2/20], Loss: 1.2362
Epoch [3/20], Loss: 0.9558
Epoch [4/20], Loss: 1.8683
Epoch [5/20], Loss: 0.7891
Epoch [6/20], Loss: 1.1763
Epoch [7/20], Loss: 1.2163
Epoch [8/20], Loss: 1.9778
Epoch [9/20], Loss: 1.2392
Epoch [10/20], Loss: 1.0851
Epoch [11/20], Loss: 1.1205
Epoch [12/20], Loss: 0.8931
Epoch [13/20], Loss: 0.6871
Epoch [14/20], Loss: 0.9240
Epoch [15/20], Loss: 0.8609
Epoch [16/20], Loss: 0.5331
Epoch [17/20], Loss: 0.7487
Epoch [18/20], Loss: 0.2863
Epoch [19/20], Loss: 1.0926
Epoch [20/20], Loss: 0.6085
The duration is 155.95753479003906


In [32]:
ts=time.time()
with torch.no_grad():
    correct=0
    total=0
    for images, labels in test_dataloader:
        images=images.to(device)
        labels=labels.to(device)
        outputs=model(images)
        _, predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+= (predicted==labels).sum().item()

    print(f"Accuracy {correct/total*100}%")

es=time.time()
print(f"Duration is {es-ts}")

Accuracy 67.56%
Duration is 1.586249828338623


In [19]:
torch.cuda.is_available()

True

In [21]:
a=torch.tensor([[1,2,3],[4,5,6]])

In [22]:
a.size(0)

2